<a href="https://colab.research.google.com/github/Abhisheksingh12345/ADT/blob/master/dev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install pyspark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plts

In [5]:
spark = SparkSession.builder.appName('dev_1').master('local[4]').getOrCreate()

In [6]:
sc = spark.sparkContext

In [7]:
table_A_path = '/content/drive/MyDrive/datasets/Table_A.csv'
table_B_path = '/content/drive/MyDrive/datasets/Table_B.csv'
table_C_path = '/content/drive/MyDrive/datasets/Table_C.csv'

In [8]:
table_A_df = spark.read.format('csv').option('inferSchema',True).option('header',True).load(table_A_path)

In [9]:
table_B_df = spark.read.format('csv').option('inferSchema',True).option('header',True).load(table_B_path)

In [10]:
table_C_df = spark.read.format('csv').option('inferSchema',True).option('header',True).load(table_C_path)

In [11]:
table_A_df.show()

+------+----------+------+----------+------------+
|cif_id|account_no|amount|department|Pride_rating|
+------+----------+------+----------+------------+
|  1222|       A01| 50000|       C2E|         3.0|
|  1353|       A03| 50000|       MOT|         1.0|
|  1262|       B01|100000|       IIT|         2.5|
|  1373|       BC2|300000|       C2E|         1.5|
+------+----------+------+----------+------------+



In [12]:
table_B_df.show()

+------+-------+-----------+---------+---------------------+
|cif_id|loan_no|loan_amount|loan_type|last_date_for_payment|
+------+-------+-----------+---------+---------------------+
|  1222|    A01|     100000|     HOME|           2023-09-31|
|  1353|    A03|     200000| PERSONAL|           2024-01-02|
|  1046|    C21|     100000|      CAR|           2023-10-11|
|  1400|    C23|     300000|     HOME|           2023-10-15|
|  1501|    D12|     400000|EDUCATION|           2024-01-02|
+------+-------+-----------+---------+---------------------+



In [13]:
table_C_df.show()

+-------+----------+------------+-----------+
|acct_no|saving_acc|current_acct|  open_date|
+-------+----------+------------+-----------+
|     12|     20000|       50000| 01-07-2023|
|     21|     30000|       60000| 23-06-2022|
|     31|     30000|        null| 2021-05-23|
|     13|    100000|        null| 02-07-2023|
|     41|      null|        null| 10-01-2023|
|     14|    320000|      600000| 05-01-2022|
+-------+----------+------------+-----------+



In [14]:
withoutscpace_tableC_df = table_C_df.na.fill("").withColumn("open_date",date_format(to_date(col("open_date"), "dd-MM-yyyy"), "yyyy-MM-dd"))

In [15]:
withoutscpace_tableC_df.show()

+-------+----------+------------+----------+
|acct_no|saving_acc|current_acct| open_date|
+-------+----------+------------+----------+
|     12|     20000|       50000|2023-07-01|
|     21|     30000|       60000|2022-06-23|
|     31|     30000|        null|      null|
|     13|    100000|        null|2023-07-02|
|     41|      null|        null|2023-01-10|
|     14|    320000|      600000|2022-01-05|
+-------+----------+------------+----------+



In [16]:
#@title Q1 merge table A and B (common column cf_id) if B not A if loan_number='D12' then delete

join_df = table_A_df.join(table_B_df, on = "cif_id", how = 'outer')
rows_to_delete_df = join_df.filter(col('account_no') == 'D12')
Q1_df = table_A_df.join (table_B_df ,on = 'cif_id', how = 'left_anti')
Q1_df.show()



+------+----------+------+----------+------------+
|cif_id|account_no|amount|department|Pride_rating|
+------+----------+------+----------+------------+
|  1262|       B01|100000|       IIT|         2.5|
|  1373|       BC2|300000|       C2E|         1.5|
+------+----------+------+----------+------------+



In [17]:
# @title Q1-part2 ->

merged_df = table_A_df.join(table_B_df, "cif_id", "inner")
result_df = merged_df.withColumn(
   "pride_rating_category",
   when(col("pride_rating") >= 3, "excellence")
   .when((col("pride_rating") > 1.5) & (col("pride_rating") < 3), "average")
   .otherwise("below average")
)
result_df.show()

+------+----------+------+----------+------------+-------+-----------+---------+---------------------+---------------------+
|cif_id|account_no|amount|department|Pride_rating|loan_no|loan_amount|loan_type|last_date_for_payment|pride_rating_category|
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+---------------------+
|  1222|       A01| 50000|       C2E|         3.0|    A01|     100000|     HOME|           2023-09-31|           excellence|
|  1353|       A03| 50000|       MOT|         1.0|    A03|     200000| PERSONAL|           2024-01-02|        below average|
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+---------------------+



In [18]:
#@title Q2 Merge table A and table B if A or B, find second highest amount for each department
from pyspark.sql.window import Window
mergeA_B_df = table_A_df.join(table_B_df, on = 'cif_id', how = "full")
mergeA_B_2df = mergeA_B_df.coalesce(4)
window_spec = Window.partitionBy("department").orderBy(col("amount").desc())
merged_df = mergeA_B_2df.withColumn("row_num", row_number().over(window_spec))
secound_highest = merged_df.filter( col('department').isNull and (col('row_num') == 2))
secound_highest.show()

+------+----------+------+----------+------------+-------+-----------+---------+---------------------+-------+
|cif_id|account_no|amount|department|Pride_rating|loan_no|loan_amount|loan_type|last_date_for_payment|row_num|
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+-------+
|  1400|      null|  null|      null|        null|    C23|     300000|     HOME|           2023-10-15|      2|
|  1222|       A01| 50000|       C2E|         3.0|    A01|     100000|     HOME|           2023-09-31|      2|
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+-------+



In [19]:
#@title Q3 Merge table A table B if A, loan no is null and last_date payroll >= 2024 then delete whole row
Q_3_df = table_A_df.join(table_B_df, on=["cif_id"], how="left_outer") \
  .filter((table_B_df["loan_no"].isNull()) & (table_B_df["last_date_for_payment"] >= "2024-01-01"))
Q_3_df.show()
Q_3_= table_A_df.join(Q_3_df.select("cif_id"), on=["cif_id"], how="left_anti")
Q_3_.show()


+------+----------+------+----------+------------+-------+-----------+---------+---------------------+
|cif_id|account_no|amount|department|Pride_rating|loan_no|loan_amount|loan_type|last_date_for_payment|
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+

+------+----------+------+----------+------------+
|cif_id|account_no|amount|department|Pride_rating|
+------+----------+------+----------+------------+
|  1222|       A01| 50000|       C2E|         3.0|
|  1353|       A03| 50000|       MOT|         1.0|
|  1262|       B01|100000|       IIT|         2.5|
|  1373|       BC2|300000|       C2E|         1.5|
+------+----------+------+----------+------------+



In [20]:
#@title Q4 Merge table A table B if A.
merge_Q4_df = table_A_df.join(table_B_df, on = 'cif_id', how = 'left')
merge_Q4_df.show()

+------+----------+------+----------+------------+-------+-----------+---------+---------------------+
|cif_id|account_no|amount|department|Pride_rating|loan_no|loan_amount|loan_type|last_date_for_payment|
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+
|  1222|       A01| 50000|       C2E|         3.0|    A01|     100000|     HOME|           2023-09-31|
|  1353|       A03| 50000|       MOT|         1.0|    A03|     200000| PERSONAL|           2024-01-02|
|  1262|       B01|100000|       IIT|         2.5|   null|       null|     null|                 null|
|  1373|       BC2|300000|       C2E|         1.5|   null|       null|     null|                 null|
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+



In [21]:
#@title Q5  For table C create a new column and add saving and current accnt
Q5_df = withoutscpace_tableC_df.withColumn('total',coalesce(col("saving_acc"), lit(0)) + coalesce(col("current_acct"), lit(0)))
# (coalesce(col("Saving_account"), lit(0)) + coalesce(col("current_account"), lit(0))).alias("total_amount"),

#    col("open_date")


Q5_df.show()

+-------+----------+------------+----------+------+
|acct_no|saving_acc|current_acct| open_date| total|
+-------+----------+------------+----------+------+
|     12|     20000|       50000|2023-07-01| 70000|
|     21|     30000|       60000|2022-06-23| 90000|
|     31|     30000|        null|      null| 30000|
|     13|    100000|        null|2023-07-02|100000|
|     41|      null|        null|2023-01-10|     0|
|     14|    320000|      600000|2022-01-05|920000|
+-------+----------+------------+----------+------+



In [22]:
#@title Q6 For table C create a new Column which have open_date in format YYYMMM
Q6_df = withoutscpace_tableC_df.withColumn("new date", date_format(col('open_date'), 'yyyyMM'))
Q6_df.show()

+-------+----------+------------+----------+--------+
|acct_no|saving_acc|current_acct| open_date|new date|
+-------+----------+------------+----------+--------+
|     12|     20000|       50000|2023-07-01|  202307|
|     21|     30000|       60000|2022-06-23|  202206|
|     31|     30000|        null|      null|    null|
|     13|    100000|        null|2023-07-02|  202307|
|     41|      null|        null|2023-01-10|  202301|
|     14|    320000|      600000|2022-01-05|  202201|
+-------+----------+------------+----------+--------+



In [23]:
#@title Q7 For table A group by deparment order of all column amonut > 1L
Q_7 = table_A_df.filter(col('amount') > 100000).groupBy(col('department')).agg(count(col('cif_id')).alias('count')).orderBy('count')
Q_7.show()

+----------+-----+
|department|count|
+----------+-----+
|       C2E|    1|
+----------+-----+

